In [ ]:
from tfaa.data_process.db_upload_prepare import DbUploadPrepare

In [ ]:
import itertools
from typing import List
import zipfile
import json
from pydantic import BaseModel
#from pydantic_mongo import AbstractRepository, ObjectIdField
from pymongo import MongoClient
from bson import ObjectId

In [ ]:
file_name = r'/home/youngsuklee/data/deribit_trade/deribit_trade_202303_partial/202303/20230301000000-20230301010000_BTC_future.zip'

In [ ]:
with zipfile.ZipFile(file_name, 'r') as zip_file:
    print(zip_file.namelist())
    x = zip_file.read('data.json')

In [ ]:
class TradeInstance(BaseModel):
    trade_seq: int
    trade_id: str 
    timestamp: int
    tick_direction: int
    price: float
    mark_price: float
    instrument_name: str
    index_price: float
    direction: str
    amount: float
    
class TradeQueryResult(BaseModel):
    trades: List[TradeInstance]
    has_more: bool

class TradeQueryResponse(BaseModel):
    jsonrpc: str
    id: int
    result: TradeQueryResult
    usIn: int
    usOut: int
    usDiff: int
    testnet: bool

In [ ]:
my_json = json.loads(x)
my_response = TradeQueryResponse(**my_json[0])

In [ ]:
client =MongoClient(host='mongodb://localhost', port=27017)

In [ ]:
all_trades = list(itertools.chain.from_iterable(x['result']['trades'] for x in my_json))


In [ ]:
all_trades_2 = all_trades

In [ ]:
my_db_name = 'my_test_db'
my_db = client[my_db_name]

In [ ]:
my_col_name = 'my_col'
my_col = my_db[my_col_name]

In [ ]:
client =MongoClient(host='mongodb://localhost', port=27017)
with client:
    with client.start_session() as session:
        with session.start_transaction():
            collection = client[my_db_name][my_col_name]
            collection.delete_many(all_trades, bypass_document_validation=True)
            #collection.insert_one(all_trades[0], validate=True)
        session.commit_transaction()
        print('SUCCESS')
#client.close()

In [ ]:
all_trades_2[0]

In [ ]:
all_trades_1[0]

In [ ]:
all_trades[-1]

In [ ]:
len(set([t['_id'] for t in all_trades]))

In [ ]:
len(all_trades)

In [ ]:
all_trades[0]

In [ ]:
my_col.insert_many(all_trades)

In [ ]:
all_trades[0]

In [ ]:
all_trades[0]

In [ ]:
ObjectId(json.dumps(all_trades[0]).encode('utf-8'))

In [ ]:
my_col.insert_one(my_json[0]['result']['trades'][1])

In [ ]:
my_col.insert_many(my_json[0]['result']['trades'])

In [ ]:
my_json[0]['result']